In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.max_colwidth = 1000
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import WhitespaceTokenizer
import string
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV, SGDClassifier
from sklearn.metrics import precision_recall_fscore_support, classification_report, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
import pickle

%load_ext autotime

# Training Set Cleaning

In [2]:
train_orig = pd.read_csv("/home/sroberts/train.csv", engine='python')

time: 59.7 s


In [3]:
train_orig[:2]

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.0,"This is so cool. It's like, 'would you want your mother to read this??' Really great idea, well done!",0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:41.987077+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.0,"Thank you!! This would make my life a lot less anxiety-inducing. Keep it up, and don't let anyone get in your way!",0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:42.870083+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4


time: 40.4 ms


In [4]:
# toxic comments
train_orig[train_orig.target >= 0.5][:2]

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.000000,0.021277,0.872340,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-29 10:50:48.488476+00,2,NaN,2006,rejected,0,0,0,1,0,0.000000,4,47
5,59859,0.666667,ur a sh*tty comment.,0.047619,0.638095,0.000000,0.333333,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:50.865549+00,2,NaN,2006,rejected,0,0,0,0,0,0.009524,0,105


time: 114 ms


### Fill NaNs with Zeros

In [5]:
train_orig.fillna(0, inplace=True)

time: 811 ms


### describe/summarize

In [6]:
desc_df = train_orig.copy()
desc_df.loc[desc_df.target >= 0.5, "toxic"] = True
desc_df.toxic.fillna(False,inplace=True)

time: 1.73 s


In [7]:
rel_col = list(set(desc_df.columns.tolist()) - set(['target','id']))
desc_df[rel_col].groupby("toxic").describe()

muslim                                              article_id                                                                                   jewish                                              identity_attack                                                             black                                              other_religion                                               parent_id                                                                              male                                                disagree                                                    female                                              homosexual_gay_or_lesbian                                              transgender                                                   white                                                 obscene                                                        other_sexual_orientation                                               sexual_explicit            \
           count      mean       std  min  25%  50%  75%  max      count           mean            std     min       25%       50%       75%       max      count      mean       std  min  25%  50%  75%  max           count      mean       std  min  25%       50%       75%  max      count      mean       std  min  25%  50%  75%  max          count      mean       std  min  25%  50%  75%  max      count          mean           std  min  25%       50%         75%        max      count      mean       std  min  25%  50%  75%  max      count      mean       std  min  25%  50%  75%    max      count      mean       std  min  25%  50%  75%  max                     count      mean       std  min  25%  50%  75%  max       count      mean       std  min  25%  50%  75%  max      count      mean       std  min  25%  50%  75%  max      count      mean       std  min  25%       50%       75%  max                    count      mean       std  min  25%  50%  75%   max           count      mean   
toxic                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
False  1660540.0  0.009336  0.089722  0.0  0.0  0.0  0.0  1.0  1660540.0  281220.733885  103935.348379  2006.0  160110.0  331961.0  366012.0  399541.0  1660540.0  0.003652  0.055521  0.0  0.0  0.0  0.0  1.0       1660540.0  0.014580  0.054317  0.0  0.0  0.000000  0.000000  0.8  1660540.0  0.005761  0.069089  0.0  0.0  0.0  0.0  1.0      1660540.0  0.001443  0.017505  0.0  0.0  0.0  0.0  1.0  1660540.0  2.128586e+06  2.612411e+06  0.0  0.0  526010.0  5360377.25  6333965.0  1660540.0  0.022545  0.129692  0.0  0.0  0.0  0.0  1.0  1660540.0  0.578480  1.854387  0.0  0.0  0.0  0.0  187.0  1660540.0  0.026821  0.149356  0.0  0.0  0.0  0.0  1.0                 1660540.0  0.004503  0.060505  0.0  0.0  0.0  0.0  1.0   1660540.0  0.001318  0.030521  0.0  0.0  0.0  0.0  1.0  1660540.0  0.009993  0.092467  0.0  0.0  0.0  0.0  1.0  1660540.0  0.005283  0.030344  0.0  0.0  0.000000  0.000000  0.7                1660540.0  0.000299  0.006870  0.0  0.0  0.0  0.0  0.75       1660540.0  0.003806   
True    144334.0  0.031420

time: 7.52 s


## lower case, stopwords, puncutation

In [8]:
#import nltk
#nltk.download('all')
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('punkt')

time: 262 µs


In [9]:
english_stopwords = set(stopwords.words('english'))
w_tokenizer = WhitespaceTokenizer()
lemmatizer = WordNetLemmatizer()

time: 11 ms


In [10]:
def clean_message(msg):
    msg = msg.lower()
    msg_tokens = nltk.word_tokenize(msg)
    clean_msg_tokens = [w for w in msg_tokens if w not in english_stopwords]
    clean_msg_tokens_puct = [w for w in clean_msg_tokens if w not in string.punctuation]
    lemmatized_token = [lemmatizer.lemmatize(w) for w in clean_msg_tokens_puct]
    return lemmatized_token

time: 4.98 ms


In [11]:
train_orig["clean"] = train_orig.comment_text.apply(lambda x: " ".join([words.strip(string.punctuation).lower()
for words in [word for word in x.split() if (word not in string.punctuation) & (word not in english_stopwords)]]))
train_orig["clean"] = train_orig["clean"].apply(lambda x: " ".join([lemmatizer.lemmatize(w) 
                                                                    for w in w_tokenizer.tokenize(x)]))

time: 4min 35s


In [12]:
train_orig['clean'][:2]

0          this cool it's like would want mother read this really great idea well done
1    thank you this would make life lot le anxiety-inducing keep up let anyone get way
Name: clean, dtype: object

time: 5.51 ms


## vectorize

In [13]:
voc_counter = Counter(" ".join(train_orig.clean).split(" "))

time: 16.4 s


In [14]:
voc_set = dict((k, v) for k, v in voc_counter.items() if v > 10)
voc_dict = {w:i for w,i in zip(voc_set, range(len(voc_set)))}

time: 145 ms


In [15]:
train_orig["clean"] = train_orig["clean"].apply(lambda x: x.split())
train_orig.loc[train_orig.target >= 0.5, "toxic"] = True
train_orig.toxic.fillna(False,inplace=True)

time: 13.2 s


In [ ]:
########
train_orig = train_orig.sample(100000)
########

In [16]:
X = np.zeros((len(train_orig), len(voc_set)), dtype = np.int8 )
for msg_idx, msg in enumerate(train_orig['clean']):
    for word in set(msg):
        if word in voc_dict:
            X[msg_idx][voc_dict[word]] = msg.count(word)
            
y = np.where(train_orig.toxic == True, 1, 0)

time: 1min 34s


In [17]:
X.shape

(1804874, 63574)

time: 2.3 ms


# Test Data

### Apply same methods to test data

In [18]:
test_orig = pd.read_csv("/home/sroberts/test.csv", engine='python')
test_orig.fillna(0, inplace=True)
test_orig['clean'] = test_orig.comment_text.apply(lambda x: " ".join([words.strip(string.punctuation).lower()
for words in [word for word in x.split() if (word not in string.punctuation) & (word not in english_stopwords)]]))
test_orig["clean"] = test_orig["clean"].apply(lambda x: " ".join([lemmatizer.lemmatize(w) 
                                                                    for w in w_tokenizer.tokenize(x)]))
test_orig["clean"] = test_orig["clean"].apply(lambda x: x.split())

X_final = np.zeros((len(test_orig), len(voc_set)), dtype = np.int8 )
for msg_idx, msg in enumerate(test_orig['clean']):
    for word in set(msg):
        if word in voc_dict:
            X_final[msg_idx][voc_dict[word]] = msg.count(word)

time: 20.7 s


# Model

## SGDClassifier

Split data into train and test

In [19]:
train_X,test_X,train_y,test_y = train_test_split(X,y,test_size=.2,random_state=2)

time: 1min 26s


In [20]:
lr = SGDClassifier(loss="log", n_jobs=-1)

#classes = np.unique(["toxic", "not_toxic"])
classes = np.unique([0, 1])

time: 797 µs


Create batches of data for training model

In [21]:
minibatches = [(train_X[:100000,:],train_y[:100000]),
               (train_X[100001:200000,:],train_y[100001:200000]),
               (train_X[200001:300000,:],train_y[200001:300000]),
               (train_X[300001:400000,:],train_y[300001:400000]),
               (train_X[400001:500000,:],train_y[400001:500000]),
               (train_X[500001:600000,:],train_y[500001:600000]),
               (train_X[600001:700000,:],train_y[600001:700000]),
               (train_X[700001:800000,:],train_y[700001:800000]),
               (train_X[800001:900000,:],train_y[800001:900000]),
               (train_X[900001:1000000,:],train_y[900001:1000000]),
               (train_X[1000001:1100000,:],train_y[1000001:1100000]),
               (train_X[1100001:1200000,:],train_y[1100001:1200000]),
               (train_X[1200001:1300000,:],train_y[1200001:1300000]),
               (train_X[1300001:1400000,:],train_y[1300001:1400000]),
               (train_X[1400001:,:],train_y[1400001:])]

time: 5.12 ms


In [22]:
for xs, ys in minibatches:
    lr.partial_fit(xs, ys, classes=classes)

time: 14min 3s


#### Save model

In [23]:
with open('SGDC.pkl', 'wb') as fid:
    pickle.dump(lr, fid)    

time: 2.07 ms


In [24]:
with open('SGDC.pkl', 'rb') as fid:
    lr = pickle.load(fid)

time: 8.84 ms


### Metrics

See how model did on our test sets

In [25]:
print(lr.score(test_X, test_y))
print(lr.intercept_)

0.9379818546990789
[-2.73390584]
time: 2min 53s


Let's look deeper at our model's accuracy

In [26]:
pred_y = lr.predict(test_X)
print(confusion_matrix(test_y, pred_y))
print(precision_score(test_y, pred_y))
print(recall_score(test_y, pred_y))
print(f1_score(test_y, pred_y))

[[330295   1895]
 [ 20492   8293]]
0.8139968590498626
0.2881014417231197
0.42557668129217663
time: 2min 25s


In [27]:
sgdc_test = lr.predict(X_final)
with open('sgdc_test.pkl', 'wb') as fid:
    pickle.dump(sgdc_test, fid) 

time: 34 s


## Use class weights

In [28]:
lr = SGDClassifier(loss="log", n_jobs=-1, class_weight={0: 0.15,1:0.85})

for xs, ys in minibatches:
    lr.partial_fit(xs, ys, classes=classes)
    
with open('SGDC_classwieghts_15_85.pkl', 'wb') as fid:
    pickle.dump(lr, fid) 

time: 16min 42s


In [29]:
with open('SGDC_classwieghts_15_85.pkl', 'rb') as fid:
    lr = pickle.load(fid)

time: 1.27 ms


In [30]:
print(lr.score(test_X, test_y))
print(lr.intercept_)
pred_y = lr.predict(test_X)
print(confusion_matrix(test_y, pred_y))
print(precision_score(test_y, pred_y))
print(recall_score(test_y, pred_y))
print(f1_score(test_y, pred_y))

0.9336269824780109
[-1.19435053]
[[320261  11929]
 [ 12030  16755]]
0.5841235532003904
0.5820739968733716
0.5830969740207764
time: 4min 52s


In [31]:
sgdc_test = lr.predict(X_final)
with open('sgdc_weights_test.pkl', 'wb') as fid:
    pickle.dump(sgdc_test, fid) 

time: 32.7 s


# MLP Classifier

In [59]:
clf = MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
classes = np.unique([0, 1])

time: 875 µs


In [60]:
for xs, ys in minibatches:
    clf.partial_fit(xs, ys, classes=classes)

time: 8min 42s


Save model

In [61]:
with open('mlpclassifier1.pkl', 'wb') as fid:
    pickle.dump(clf, fid)

time: 7.01 ms


In [62]:
with open('mlpclassifier1.pkl', 'rb') as fid:
    clf = pickle.load(fid)

time: 19.1 ms


### Metrics

In [63]:
print(clf.score(test_X, test_y))
print(clf.coefs_)
print(clf.intercepts_)

0.9202576355703304
[array([[ 0.01340488,  0.00452412,  0.00417414,  0.01051082, -0.00394437],
       [-0.00850056, -0.00608622, -0.00322385, -0.00247091,  0.00090503],
       [ 0.00274912,  0.00378436, -0.00183613,  0.00818325, -0.00757463],
       ...,
       [ 0.004894  , -0.00787085, -0.00903129, -0.00810711,  0.00217808],
       [-0.00250236, -0.00478914,  0.0044798 , -0.005729  , -0.00092925],
       [-0.00548877, -0.00639406,  0.00365675,  0.00561296,  0.00538749]]), array([[-0.82510757,  0.24408555],
       [ 0.01808173, -0.88630158],
       [-0.53167537, -0.34125329],
       [-0.68018901, -0.40900986],
       [ 0.09482703, -0.84904143]]), array([[-0.94677255],
       [ 0.72207098]])]
[array([0.06833846, 0.01246148, 0.05163452, 0.06098633, 0.034911  ]), array([ 0.40745462, -0.06732792]), array([-2.2325339])]
time: 2min 29s


In [64]:
pred_y = clf.predict(test_X)
print(confusion_matrix(test_y, pred_y))
print(precision_score(test_y, pred_y))
print(recall_score(test_y, pred_y))
print(f1_score(test_y, pred_y))

[[332190      0]
 [ 28785      0]]
0.0
0.0
0.0
time: 2min 21s


/home/sroberts/anaconda3/envs/toxic/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/sroberts/anaconda3/envs/toxic/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [65]:
mlp_test = clf.predict(X_final)
with open('mlp_test.pkl', 'wb') as fid:
    pickle.dump(mlp_test, fid) 

time: 34.2 s


## Use class weights

In [ ]:
# new class
class MLPClassifierOverride(MLPClassifier):
# Overriding _init_coef method
def _init_coef(self, fan_in, fan_out):
    if self.activation == 'logistic':
        init_bound = np.sqrt(2. / (fan_in + fan_out))
    elif self.activation in ('identity', 'tanh', 'relu'):
        init_bound = np.sqrt(6. / (fan_in + fan_out))
    else:
        raise ValueError("Unknown activation function %s" %
                         self.activation)
    coef_init = ### place your initial values for coef_init here

    intercept_init = ### place your initial values for intercept_init here

    return coef_init, intercept_init

In [ ]:
mlp_test = clf.predict(X_final)
with open('mlp_weights_test.pkl', 'wb') as fid:
    pickle.dump(mlp_test, fid) 

# Naive Bayes

In [32]:
nb = MultinomialNB()

time: 605 µs


In [33]:
for xs, ys in minibatches:
    nb.partial_fit(xs, ys, classes=classes)

time: 15min 20s


Save model

In [34]:
with open('naivebayes.pkl', 'wb') as fid:
    pickle.dump(nb, fid) 

time: 5.04 ms


In [35]:
with open('naivebayes.pkl', 'rb') as fid:
    nb = pickle.load(fid)

time: 11 ms


### Metrics

In [36]:
print(nb.score(test_X, test_y))
print(nb.intercept_)
print(nb.coef_)
print(nb.feature_count_)
print(nb.class_count_)

0.8938181314495464
[-2.52542396]
[[ -5.67466923  -9.61271434  -6.354443   ... -15.00634189 -13.9077296
  -14.31319471]]
[[1.21552e+05 3.08200e+03 6.26540e+04 ... 1.00000e+01 9.00000e+00
  9.00000e+00]
 [1.12890e+04 2.19000e+02 5.72000e+03 ... 0.00000e+00 2.00000e+00
  1.00000e+00]]
[1328339.  115546.]
time: 2min 29s


In [37]:
pred_y = nb.predict(test_X)
print(confusion_matrix(test_y, pred_y))
print(precision_score(test_y, pred_y))
print(recall_score(test_y, pred_y))
print(f1_score(test_y, pred_y))

[[304445  27745]
 [ 10584  18201]]
0.3961389457188874
0.6323084940072955
0.4871070907655458
time: 2min 20s


In [38]:
nb_test = nb.predict(X_final)
with open('nb_test.pkl', 'wb') as fid:
    pickle.dump(nb_test, fid) 

time: 33.9 s


## Use class weights

In [39]:
nb= MultinomialNB(alpha= 0.1, fit_prior = True, class_prior = [0.15, 0.85])
for xs, ys in minibatches:
    nb.partial_fit(xs, ys, classes=classes)

time: 15min 16s


In [40]:
with open('naivebayes_weights_class_weights.pkl', 'wb') as fid:
    pickle.dump(nb, fid) 

time: 3.93 ms


In [41]:
with open('naivebayes_weights_class_weights.pkl', 'rb') as fid:
    nb = pickle.load(fid)

time: 6.3 ms


In [42]:
print(nb.score(test_X, test_y))
print(nb.intercept_)
print(nb.coef_)
print(nb.feature_count_)
print(nb.class_count_)

0.504813352725258
[-0.16251893]
[[ -5.65720389  -9.59926858  -6.33705526 ... -17.29138192 -14.24685949
  -14.89348665]]
[[1.21552e+05 3.08200e+03 6.26540e+04 ... 1.00000e+01 9.00000e+00
  9.00000e+00]
 [1.12890e+04 2.19000e+02 5.72000e+03 ... 0.00000e+00 2.00000e+00
  1.00000e+00]]
[1328339.  115546.]
time: 2min 24s


In [43]:
pred_y = nb.predict(test_X)
print(confusion_matrix(test_y, pred_y))
print(precision_score(test_y, pred_y))
print(recall_score(test_y, pred_y))
print(f1_score(test_y, pred_y))

[[155933 176257]
 [  2493  26292]]
0.12980562728031242
0.913392391870766
0.22730770228327876
time: 2min 21s


In [44]:
nb_test = nb.predict(X_final)
with open('nb_weights_test.pkl', 'wb') as fid:
    pickle.dump(nb_test, fid) 

time: 34 s


# NB FIX

In [66]:
nb= MultinomialNB(alpha= 0.1, fit_prior = True, class_prior = [0.2, 0.8])
for xs, ys in minibatches:
    nb.partial_fit(xs, ys, classes=classes)
    
with open('naivebayes_weights_class_weights_2_8.pkl', 'wb') as fid:
    pickle.dump(nb, fid) 

print(nb.score(test_X, test_y))
print(nb.intercept_)
print(nb.coef_)
print(nb.feature_count_)
print(nb.class_count_)

pred_y = nb.predict(test_X)
print(confusion_matrix(test_y, pred_y))
print(precision_score(test_y, pred_y))
print(recall_score(test_y, pred_y))
print(f1_score(test_y, pred_y))


0.5349345522543112
[-0.22314355]
[[ -5.65720389  -9.59926858  -6.33705526 ... -17.29138192 -14.24685949
  -14.89348665]]
[[1.21552e+05 3.08200e+03 6.26540e+04 ... 1.00000e+01 9.00000e+00
  9.00000e+00]
 [1.12890e+04 2.19000e+02 5.72000e+03 ... 0.00000e+00 2.00000e+00
  1.00000e+00]]
[1328339.  115546.]
[[167045 165145]
 [  2732  26053]]
0.13626188558457725
0.9050894563140525
0.23686375765400053
time: 19min 40s


In [ ]:
nb= MultinomialNB(alpha= 0.1, fit_prior = True, class_prior = [0.25, 0.75])
for xs, ys in minibatches:
    nb.partial_fit(xs, ys, classes=classes)
    
#with open('naivebayes_weights_class_weights_2_8.pkl', 'wb') as fid:
#    pickle.dump(nb, fid) 

print(nb.score(test_X, test_y))

pred_y = nb.predict(test_X)
print(confusion_matrix(test_y, pred_y))
print(precision_score(test_y, pred_y))
print(recall_score(test_y, pred_y))
print(f1_score(test_y, pred_y))

In [ ]:
nb= MultinomialNB(alpha= 0.1, fit_prior = True, class_prior = [0.3, 0.7])
for xs, ys in minibatches:
    nb.partial_fit(xs, ys, classes=classes)
    
#with open('naivebayes_weights_class_weights_2_8.pkl', 'wb') as fid:
#    pickle.dump(nb, fid) 

print(nb.score(test_X, test_y))

pred_y = nb.predict(test_X)
print(confusion_matrix(test_y, pred_y))
print(precision_score(test_y, pred_y))
print(recall_score(test_y, pred_y))
print(f1_score(test_y, pred_y))

In [ ]:
nb= MultinomialNB(alpha= 0.1, fit_prior = True, class_prior = [0.35, 0.65])
for xs, ys in minibatches:
    nb.partial_fit(xs, ys, classes=classes)
    
#with open('naivebayes_weights_class_weights_2_8.pkl', 'wb') as fid:
#    pickle.dump(nb, fid) 

print(nb.score(test_X, test_y))

pred_y = nb.predict(test_X)
print(confusion_matrix(test_y, pred_y))
print(precision_score(test_y, pred_y))
print(recall_score(test_y, pred_y))
print(f1_score(test_y, pred_y))

# RandomForestClassifier

In [45]:
rf = RandomForestClassifier(warm_start=True, n_estimators=100, max_depth=8, n_jobs=-1)

time: 781 µs


In [46]:
for xs, ys in minibatches:
    rf.fit(xs,ys)
    rf.n_estimators += 1

time: 8min 28s


save model

In [47]:
with open('randomforest.pkl', 'wb') as fid:
    pickle.dump(rf, fid)

time: 8.07 ms


In [48]:
with open('randomforest.pkl', 'rb') as fid:
    rf = pickle.load(fid)

time: 14.3 ms


In [49]:
print(rf.score(test_X, test_y))
print(rf.estimators_)
print(rf.n_features_)
print(rf.n_outputs_)
print(rf.feature_importances_)

0.9202576355703304
[DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=8,
                       max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=347702793, splitter='best'), DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=8,
                       max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=1449612355, splitter='best'), DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=8,
                       max_features='auto', max_leaf_nodes=None,
                     

In [50]:
pred_y = rf.predict(test_X)
print(confusion_matrix(test_y, pred_y))
print(precision_score(test_y, pred_y))
print(recall_score(test_y, pred_y))
print(f1_score(test_y, pred_y))

[[332190      0]
 [ 28785      0]]
0.0
0.0
0.0
time: 1min 55s


/home/sroberts/anaconda3/envs/toxic/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/sroberts/anaconda3/envs/toxic/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [51]:
rf_test = rf.predict(X_final)
with open('rf_test.pkl', 'wb') as fid:
    pickle.dump(rf_test, fid) 

time: 22.6 s


## Use class weights

In [52]:
rf = RandomForestClassifier(warm_start=True, n_estimators=100, max_depth=8, n_jobs=-1, class_weight={0: 0.15,1:0.85})

time: 1.53 ms


In [53]:
for xs, ys in minibatches:
    rf.fit(xs,ys)
    rf.n_estimators += 1

time: 8min 47s


save model

In [54]:
with open('randomforest_classweights.pkl', 'wb') as fid:
    pickle.dump(rf, fid)

time: 8.7 ms


In [55]:
with open('randomforest_classweights.pkl', 'rb') as fid:
    rf = pickle.load(fid)

time: 13.2 ms


In [56]:
print(rf.score(test_X, test_y))
print(rf.estimators_)
print(rf.n_features_)
print(rf.n_outputs_)
print(rf.feature_importances_)

0.9202576355703304
[DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=8,
                       max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=1498810984, splitter='best'), DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=8,
                       max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=1481232554, splitter='best'), DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=8,
                       max_features='auto', max_leaf_nodes=None,
                    

In [57]:
pred_y = rf.predict(test_X)
print(confusion_matrix(test_y, pred_y))
print(precision_score(test_y, pred_y))
print(recall_score(test_y, pred_y))
print(f1_score(test_y, pred_y))

[[332190      0]
 [ 28785      0]]
0.0
0.0
0.0
time: 1min 55s


/home/sroberts/anaconda3/envs/toxic/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/sroberts/anaconda3/envs/toxic/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [58]:
rf_test = rf.predict(X_final)
with open('rf_weights_test.pkl', 'wb') as fid:
    pickle.dump(rf_test, fid) 

time: 22.6 s


# RF FIX

In [70]:
rf = RandomForestClassifier(warm_start=True, n_jobs=-1, class_weight='balanced')

for xs, ys in minibatches:
    rf.fit(xs,ys)
    rf.n_estimators += 1

with open('randomforest_classweights_25.pkl', 'wb') as fid:
    pickle.dump(rf, fid)
    
print(rf.score(test_X, test_y))
print(rf.estimators_)
print(rf.n_features_)
print(rf.n_outputs_)
print(rf.feature_importances_)

pred_y = rf.predict(test_X)
print(confusion_matrix(test_y, pred_y))
print(precision_score(test_y, pred_y))
print(recall_score(test_y, pred_y))
print(f1_score(test_y, pred_y))

/home/sroberts/anaconda3/envs/toxic/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/sroberts/anaconda3/envs/toxic/lib/python3.7/site-packages/sklearn/ensemble/forest.py:504: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the class_weight parameter.
  warn('class_weight presets "balanced" or "balanced_subsample" are '
/home/sroberts/anaconda3/envs/toxic/lib/python3.7/site-packages/sklearn/ensemble/forest.py:504: UserWarning: class_weight presets "balanced" or "bal

/home/sroberts/anaconda3/envs/toxic/lib/python3.7/site-packages/sklearn/ensemble/forest.py:504: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the class_weight parameter.
  warn('class_weight presets "balanced" or "balanced_subsample" are '


0.9325715077221414
[DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=222467800, splitter='best'), DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=243281019, splitter='best'), DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features='auto', max_leaf_nodes=None,
             

[[330917   1273]
 [ 23067   5718]]
0.817908739808325
0.1986451276706618
0.31965563506261174
time: 39min 16s


In [72]:
rf = RandomForestClassifier(warm_start=True, n_jobs=-1, class_weight='balanced')

for xs, ys in minibatches:
    rf.fit(xs,ys)
    rf.n_estimators += 1

with open('randomforest_classweights_25.pkl', 'wb') as fid:
    pickle.dump(rf, fid)
    
print(rf.score(test_X, test_y))
print(rf.estimators_)
print(rf.n_features_)
print(rf.n_outputs_)
print(rf.feature_importances_)

pred_y = rf.predict(test_X)
print(confusion_matrix(test_y, pred_y))
print(precision_score(test_y, pred_y))
print(recall_score(test_y, pred_y))
print(f1_score(test_y, pred_y))

/home/sroberts/anaconda3/envs/toxic/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/sroberts/anaconda3/envs/toxic/lib/python3.7/site-packages/sklearn/ensemble/forest.py:504: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the class_weight parameter.
  warn('class_weight presets "balanced" or "balanced_subsample" are '
/home/sroberts/anaconda3/envs/toxic/lib/python3.7/site-packages/sklearn/ensemble/forest.py:504: UserWarning: class_weight presets "balanced" or "bal

/home/sroberts/anaconda3/envs/toxic/lib/python3.7/site-packages/sklearn/ensemble/forest.py:504: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the class_weight parameter.
  warn('class_weight presets "balanced" or "balanced_subsample" are '


0.9321892097790706
[DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=1877071902, splitter='best'), DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=1548640274, splitter='best'), DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features='auto', max_leaf_nodes=None,
           

[[331084   1106]
 [ 23372   5413]]
0.8303420770056757
0.18804933124891438
0.30665080444142306
time: 39min 31s


In [71]:
rf = RandomForestClassifier(warm_start=True, n_jobs=-1, class_weight={0: 1, 1: 15})

for xs, ys in minibatches:
    rf.fit(xs,ys)
    rf.n_estimators += 1

with open('randomforest_classweights_25.pkl', 'wb') as fid:
    pickle.dump(rf, fid)
    
print(rf.score(test_X, test_y))
print(rf.estimators_)
print(rf.n_features_)
print(rf.n_outputs_)
print(rf.feature_importances_)

pred_y = rf.predict(test_X)
print(confusion_matrix(test_y, pred_y))
print(precision_score(test_y, pred_y))
print(recall_score(test_y, pred_y))
print(f1_score(test_y, pred_y))

/home/sroberts/anaconda3/envs/toxic/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9323388046263592
[DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=2078545837, splitter='best'), DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=234220083, splitter='best'), DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features='auto', max_leaf_nodes=None,
            

[[331095   1095]
 [ 23329   5456]]
0.8328499465730422
0.18954316484280007
0.30880688249943405
time: 38min 42s


In [ ]:
rf = RandomForestClassifier(warm_start=True, n_estimators=400, max_depth=8, n_jobs=-1, class_weight={0: 1, 1: 50})

for xs, ys in minibatches:
    rf.fit(xs,ys)
    rf.n_estimators += 1

#with open('randomforest_classweights_25.pkl', 'wb') as fid:
#    pickle.dump(rf, fid)
    
print(rf.score(test_X, test_y))
#print(rf.estimators_)
#print(rf.n_features_)
#print(rf.n_outputs_)
#print(rf.feature_importances_)

pred_y = rf.predict(test_X)
print(confusion_matrix(test_y, pred_y))
print(precision_score(test_y, pred_y))
print(recall_score(test_y, pred_y))
print(f1_score(test_y, pred_y))

In [ ]:
rf = RandomForestClassifier(warm_start=True, n_estimators=400, max_depth=8, n_jobs=-1, class_weight={0: 1, 1: 30})

for xs, ys in minibatches:
    rf.fit(xs,ys)
    rf.n_estimators += 1

#with open('randomforest_classweights_25.pkl', 'wb') as fid:
#    pickle.dump(rf, fid)
    
print(rf.score(test_X, test_y))
#print(rf.estimators_)
#print(rf.n_features_)
#print(rf.n_outputs_)
#print(rf.feature_importances_)

pred_y = rf.predict(test_X)
print(confusion_matrix(test_y, pred_y))
print(precision_score(test_y, pred_y))
print(recall_score(test_y, pred_y))
print(f1_score(test_y, pred_y))